## convert the csv file of MNF data extracted from the Data Trawler to netCDF

In [2]:
import pandas as pd
import datetime as dt
from pathlib import Path
import os
import sys
# include the IMOS python toolbox to assist with attributes
sys.path.insert(1, '/oa-decadal-climate/work/observations/oceanobs_data/UOT/programs/data-services/lib/python/')
import numpy as np
from netCDF4 import Dataset, date2num
from generate_netcdf_att import generate_netcdf_att, get_imos_parameter_info

In [3]:
# now let's read in the csv CTD data from MNF
#MNF_data_path ='/tube1/cha674/CARS_2022_ancillary_data/MNF/'
MNF_data_path = '/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/'
filelist = Path(MNF_data_path).rglob('*.csv')# read the csv file

#MNF_CTD_dataframe = pandas.read_csv(os.path.join(file_path,data_dirs[0],file_names[0]))


In [4]:
MNF_data_path

'/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/'

In [5]:
filn = str(next(filelist))
filn

'/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/NCMIGBR_CTDdata.csv'

In [6]:
df = pd.read_csv(filn, on_bad_lines='warn', low_memory=False)
# in this file, there are some data from SOELA with an extra comma in the 
# PROJECT NAME field and we are skipping them for now

Skipping line 1080051: expected 21 fields, saw 22
Skipping line 1080052: expected 21 fields, saw 22
Skipping line 1080053: expected 21 fields, saw 22
Skipping line 1080054: expected 21 fields, saw 22
Skipping line 1080055: expected 21 fields, saw 22
Skipping line 1080056: expected 21 fields, saw 22
Skipping line 1080057: expected 21 fields, saw 22
Skipping line 1080058: expected 21 fields, saw 22
Skipping line 1080059: expected 21 fields, saw 22
Skipping line 1080060: expected 21 fields, saw 22
Skipping line 1080061: expected 21 fields, saw 22
Skipping line 1080062: expected 21 fields, saw 22
Skipping line 1080063: expected 21 fields, saw 22
Skipping line 1080064: expected 21 fields, saw 22
Skipping line 1080065: expected 21 fields, saw 22
Skipping line 1080066: expected 21 fields, saw 22
Skipping line 1080067: expected 21 fields, saw 22
Skipping line 1080068: expected 21 fields, saw 22
Skipping line 1080069: expected 21 fields, saw 22
Skipping line 1080070: expected 21 fields, saw 22


In [7]:
df.columns


Index(['SURVEY_NAME', 'STATION', 'START_TIME', 'END_TIME', 'MIN_DEPTH',
       'MAX_DEPTH', 'BOTTOM_DEPTH', 'BOTTOM_LAT', 'BOTTOM_LON', 'END_LAT',
       'END_LON', 'START_LAT', 'START_LON', 'PROJECT_NAME', 'MARLIN_ID',
       'MARLIN_UUID', 'PRESSURE', 'SALINITY', 'SALINITY_QC', 'TEMPERATURE',
       'TEMPERATURE_QC'],
      dtype='object')

In [62]:
# set up dictionaries to map the cast dimensioned names to the MNF names
imosnames = ['TEMP','PRES_REL','PSAL']
mnfnames = ['TEMPERATURE','PRESSURE','SALINITY']
vardict = dict(zip(mnfnames,imosnames))
vardict2 = dict(zip(imosnames,mnfnames))
print(vardict)
imosglobnames = ['cruise','station','project','site_code','unique_code' ]
mnfglobnames = ['SURVEY_NAME','STATION','PROJECT_NAME','MARLIN_ID','MARLIN_UUID' ]
globdict = dict(zip(mnfglobnames,imosglobnames))
print(globdict)

{'TEMPERATURE': 'TEMP', 'PRESSURE': 'PRES_REL', 'SALINITY': 'PSAL'}
{'SURVEY_NAME': 'cruise', 'STATION': 'station', 'PROJECT_NAME': 'project', 'MARLIN_ID': 'site_code', 'MARLIN_UUID': 'unique_code'}


In [16]:
df

,SURVEY_NAME,STATION,START_TIME,END_TIME,MIN_DEPTH,MAX_DEPTH,BOTTOM_DEPTH,BOTTOM_LAT,BOTTOM_LON,END_LAT,...,START_LAT,START_LON,PROJECT_NAME,MARLIN_ID,MARLIN_UUID,PRESSURE,SALINITY,SALINITY_QC,TEMPERATURE,TEMPERATURE_QC
0,FR198503,3,19-JUN-1985 15:52:00,19-JUN-1985 15:52:00,0.0,992.0,1015.0,-20.23883,153.14033,-20.23883,...,-20.23883,153.14033,Mesoscale and Continental Shelf Oceanography 1...,4793.0,34f8dc34-dbe9-4edd-8ae1-92315ab54491,4.0,35.223000,32.0,24.158202,32.0
1,FR198503,3,19-JUN-1985 15:52:00,19-JUN-1985 15:52:00,0.0,992.0,1015.0,-20.23883,153.14033,-20.23883,...,-20.23883,153.14033,Mesoscale and Continental Shelf Oceanography 1...,4793.0,34f8dc34-dbe9-4edd-8ae1-92315ab54491,6.0,35.223000,32.0,24.158202,32.0
2,FR198503,3,19-JUN-1985 15:52:00,19-JUN-1985 15:52:00,0.0,992.0,1015.0,-20.23883,153.14033,-20.23883,...,-20.23883,153.14033,Mesoscale and Continental Shelf Oceanography 1...,4793.0,34f8dc34-dbe9-4edd-8ae1-92315ab54491,8.0,35.223000,32.0,24.160202,32.0
3,FR198503,3,19-JUN-1985 15:52:00,19-JUN-1985 15:52:00,0.0,992.0,1015.0,-20.23883,153.14033,-20.23883,...,-20.23883,153.14033,Mesoscale and Continental Shelf Oceanography 1...,4793.0,34f8dc34-dbe9-4edd-8ae1-92315ab54491,10.0,35.222000,32.0,24.165200,32.0
4,FR198503,3,19-JUN-1985 15:52:00,19-JUN-1985 15:52:00,0.0,992.0,1015.0,-20.23883,153.14033,-20.23883,...,-20.23883,153.14033,Mesoscale and Continental Shelf Oceanography 1...,4793.0,34f8dc34-dbe9-4edd-8ae1-92315ab54491,12.0,35.223000,32.0,24.168199,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274316,SS2013_V05,28,30-AUG-2013 18:44:55,30-AUG-2013 21:52:03,0.0,4790.0,4699.8,-26.27758,155.65561,-26.27665,...,-26.27661,155.66343,Sustained Monitoring of the East Australian Cu...,14346.0,e7a98393-b4b3-636a-e043-08114f8c25c0,4786.0,34.713364,32.0,1.218980,32.0
1274317,SS2013_V05,28,30-AUG-2013 18:44:55,30-AUG-2013 21:52:03,0.0,4790.0,4699.8,-26.27758,155.65561,-26.27665,...,-26.27661,155.66343,Sustained Monitoring of the East Australian Cu...,14346.0,e7a98393-b4b3-636a-e043-08114f8c25c0,4787.0,34.713409,32.0,1.219064,32.0
1274318,SS2013_V05,28,30-AUG-2013 18:44:55,30-AUG-2013 21:52:03,0.0,4790.0,4699.8,-26.27758,155.65561,-26.27665,...,-26.27661,155.66343,Sustained Monitoring of the East Australian Cu...,14346.0,e7a98393-b4b3-636a-e043-08114f8c25c0,4788.0,34.713345,32.0,1.219234,32.0
1274319,SS2013_V05,28,30-AUG-2013 18:44:55,30-AUG-2013 21:52:03,0.0,4790.0,4699.8,-26.27758,155.65561,-26.27665,...,-26.27661,155.66343,Sustained Monitoring of the East Australian Cu...,14346.0,e7a98393-b4b3-636a-e043-08114f8c25c0,4789.0,34.713371,32.0,1.219380,32.0


## Need to group the data by profile, one netcdf file per cast

In [20]:
dfgroup = df.groupby(['SURVEY_NAME','STATION'])

In [28]:
dfgroup.groups

{('FR198503', 3): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], ('FR198503', 4): [495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, ...], ('FR198503', 5): [748, 749, 750, 751, 752, 753, 754, 755, 756

In [35]:
dfgroup.nunique()

START_TIME  END_TIME  MIN_DEPTH  MAX_DEPTH  BOTTOM_DEPTH  \
SURVEY_NAME STATION                                                             
FR198503    3                 1         1          1          1             1   
            4                 1         1          1          1             1   
            5                 1         1          1          1             1   
            6                 1         1          1          1             1   
            7                 1         1          1          1             1   
...                         ...       ...        ...        ...           ...   
SS2013_V05  24                1         1          1          1             1   
            25                1         1          1          1             1   
            26                1         1          1          1             1   
            27                1         1          1          1             1   
            28                1         1          1          1             1   

                     BOTTOM_LAT  BOTTOM_LON  END_LAT  END_LON  START_LAT  \
SURVEY_NAME STATION                                                        
FR198503    3                 1           1        1        1          1   
            4                 1           1        1        1          1   
            5                 1           1        1        1          1   
            6                 1           1        1        1          1   
            7                 1           1        1        1          1   
...                         ...         ...      ...      ...        ...   
SS2013_V05  24                1           1        1        1          1   
            25                1           1        1        1          1   
            26                1           1        1        1          1   
            27                1           1        1        1          1   
            28                1           1        1        1          1   

                     START_LON  PROJECT_NAME  MARLIN_ID  MARLIN_UUID  \
SURVEY_NAME STATION                                                    
FR198503    3                1             1          1            1   
            4                1             1          1            1   
            5                1             1          1            1   
            6                1             1          1            1   
            7                1             1          1            1   
...                        ...           ...        ...          ...   
SS2013_V05  24               1             1          1            1   
            25               1             1          1            1   
            26               1             1          1            1   
            27               1             1          1            1   
            28               1             1          1            1   

                     PRESSURE  SALINITY  SALINITY_QC  TEMPERATURE  \
SURVEY_NAME STATION                                                 
FR198503    3             495       307            1          451   
            4             253       200            1          239   
            5             102        70            1           97   
            6             178       144            1          171   
            7             211       180            1          205   
...                       ...       ...          ...          ...   
SS2013_V05  24           4530      3662            1         4514   
            25           4704      3824            1         4677   
            26           4788      3790            1         4772   
            27           4786      3752            1         4765   
            28           4513      3682            1         4492   

                     TEMPERATURE_QC  
SURVEY_NAME STATION                  
FR198503    3                     1  
            4                

In [46]:
for name, cast in dfgroup:
    nn = name[0] + str(name[1]).zfill(3)
    outf = os.path.join(os.path.dirname(filn), nn) + '.nc'
    print(outf)

/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503003.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503004.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503005.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503006.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503007.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503008.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503011.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503012.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503013.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503014.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503015.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/FR198503016.nc
/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/F

In [66]:
cast['BOTTOM_DEPTH'].unique()

array([4699.8])

## FLAGS

    Will do a flag conversion to IMOS flags, final column
    State        Unsigned Byte          Signed Byte         IMOS equivalent
    Good         0 ≤ flag ≤ 63          0 ≤ flag ≤ 63       1 
    Suspect      64 ≤ flag ≤ 127        64 ≤ flag ≤ 127     3 
    Bad          128 ≤ flag ≤ 191       -128 ≤ flag ≤ -65   4
    No QC        192 ≤ flag ≤ 255       -64 ≤ flag ≤ -1     0
    

In [89]:
dt.datetime.strptime(data['START_TIME'].unique().item(), '%d-%b-%Y %H:%M:%S')

datetime.datetime(1985, 6, 19, 15, 52)

In [90]:
data['START_TIME'].unique().item()

'19-JUN-1985 15:52:00'

## Will need to do a pressure to depth conversion

In [11]:
import gsw

In [12]:
gsw.z_from_p(300,-27)

-297.81137724942045

In [128]:
# now let's try the netcdf creation
MNF_data_path = '/oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/'
filelist = Path(MNF_data_path).rglob('*.csv')# read the csv file

# each MNF file contains multiple casts
for filn in filelist:   
    # read the data:
    df = pd.read_csv(filn, on_bad_lines='warn', low_memory=False)
    
    # the parameter names vary from file to file
    dfgroup = df.groupby(['SURVEY_NAME','STATION'])
    
    for deployment, data in dfgroup:
        # set up the output file name:
        nn = deployment[0] + str(deployment[1]).zfill(3)
        outfile = os.path.join(os.path.dirname(filn), nn) + '.nc'

        # read a file with the global attributes included and the nc configuration file
        conf_file_generic = '/tube1/cow074/Documents/cars-v2/notebooks/generate_nc_file_attMNF'

        # get the other coordinates
        lat = data['START_LAT'].unique()
        #check for more than one lat/lon/time value, skip
        if len(lat) > 1:
            print(['Multiple location information: ' + outfile
            ])
            continue
        lon = data['START_LON'].unique()
        time = dt.datetime.strptime(data['START_TIME'].unique().item(), '%d-%b-%Y %H:%M:%S')
        botdepth = data['BOTTOM_DEPTH'].unique()

        # get the coordinate/depth dimension
        pres = data['PRESSURE']
        depth = -gsw.z_from_p(pres,lat)
        
        # create a netcdf object and write depth,time,lat,long to it:
        with Dataset(outfile, 'w', format='NETCDF4') as output_netcdf_obj:
            # first create our DEPTH dimension and variable
            output_netcdf_obj.createDimension("DEPTH", depth.size)
            output_netcdf_obj.createVariable("DEPTH", "f", "DEPTH")
            output_netcdf_obj['DEPTH'][:] = depth
            # and lat/lon/time vars which come from the header in the csv file:
            output_netcdf_obj.createVariable('TIME','d', fill_value=get_imos_parameter_info('TIME', '_FillValue'))

            output_netcdf_obj.createVariable("LATITUDE", "f", fill_value=get_imos_parameter_info('LATITUDE', '_FillValue'))
            output_netcdf_obj['LATITUDE'][:] = lat
            output_netcdf_obj.createVariable("LONGITUDE", "f", fill_value=get_imos_parameter_info('LONGITUDE', '_FillValue'))
            output_netcdf_obj['LONGITUDE'][:] = lon  
            output_netcdf_obj.createVariable("BOT_DEPTH", "f", fill_value=get_imos_parameter_info('BOT_DEPTH', '_FillValue'))
            output_netcdf_obj['BOT_DEPTH'][:] = botdepth  

            # now all the other variables
            for group in vardict:
                dat = np.ma.masked_invalid(data[group])
                if 'PRESSURE' not in group:
                    flag = np.ma.masked_invalid(data[group + '_QC'])
                else:
                    flag = np.ma.masked_invalid(dat)
                    flag[:] = np.nan
                # map flag values to IMOS flags
                flag[np.logical_and(flag>= 0, flag<=63)] = 1
                flag[np.logical_and(flag>= 64, flag<=127)] = 3
                flag[np.logical_and(flag>= 128, flag<=191)] = 4
                flag[np.logical_and(flag>= 192, flag<=255)] = 0
                name = vardict[group]
                #create the variable & QC variable:
                output_netcdf_obj.createVariable(name, "f", ["DEPTH"], 
                            fill_value=get_imos_parameter_info(name, '_FillValue'))
                output_netcdf_obj.createVariable(name + '_quality_control', "b", ["DEPTH"], 
                            fill_value=99)

                # output the data
                output_netcdf_obj[name][:] = dat
                output_netcdf_obj[name + '_quality_control'][:] = flag
                
            #generate all the attributes for the variables & the global attributes too
            generate_netcdf_att(output_netcdf_obj, conf_file_generic, conf_file_point_of_truth=True)

            #now we can output the date/time value (need to have the generation of attributes done first)
            time_val_dateobj = date2num(time, output_netcdf_obj['TIME'].units, output_netcdf_obj['TIME'].calendar)
            output_netcdf_obj['TIME'][:] = time_val_dateobj

            #global attributes from header
            for value in globdict:
                var = data[value].unique().item()
                setattr(output_netcdf_obj, globdict[value], var) 
            # do the date/time stamps individually
            setattr(output_netcdf_obj, 'date_created', dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"))

Skipping line 1080051: expected 21 fields, saw 22
Skipping line 1080052: expected 21 fields, saw 22
Skipping line 1080053: expected 21 fields, saw 22
Skipping line 1080054: expected 21 fields, saw 22
Skipping line 1080055: expected 21 fields, saw 22
Skipping line 1080056: expected 21 fields, saw 22
Skipping line 1080057: expected 21 fields, saw 22
Skipping line 1080058: expected 21 fields, saw 22
Skipping line 1080059: expected 21 fields, saw 22
Skipping line 1080060: expected 21 fields, saw 22
Skipping line 1080061: expected 21 fields, saw 22
Skipping line 1080062: expected 21 fields, saw 22
Skipping line 1080063: expected 21 fields, saw 22
Skipping line 1080064: expected 21 fields, saw 22
Skipping line 1080065: expected 21 fields, saw 22
Skipping line 1080066: expected 21 fields, saw 22
Skipping line 1080067: expected 21 fields, saw 22
Skipping line 1080068: expected 21 fields, saw 22
Skipping line 1080069: expected 21 fields, saw 22
Skipping line 1080070: expected 21 fields, saw 22


/tmp/ipykernel_1167/922547137.py:74: RuntimeWarning: invalid value encountered in cast
  output_netcdf_obj[name + '_quality_control'][:] = flag


['Multiple location information: /oa-decadal-climate/work/observations/CARSv2_ancillary/MNF_GBR/SS199706005.nc']


In [127]:
data

,SURVEY_NAME,STATION,START_TIME,END_TIME,MIN_DEPTH,MAX_DEPTH,BOTTOM_DEPTH,BOTTOM_LAT,BOTTOM_LON,END_LAT,...,START_LAT,START_LON,PROJECT_NAME,MARLIN_ID,MARLIN_UUID,PRESSURE,SALINITY,SALINITY_QC,TEMPERATURE,TEMPERATURE_QC
1084992,SS199706,5,11-MAY-1997 09:54:00,11-MAY-1997 10:43:00,0.0,990.0,4600.0,-14.69300,151.3260,-14.69300,...,-14.69300,151.3260,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,2.0,34.915001,32.0,26.986000,32.0
1084993,SS199706,5,11-MAY-1997 09:54:00,11-MAY-1997 10:43:00,0.0,990.0,4600.0,-14.69300,151.3260,-14.69300,...,-14.69300,151.3260,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,4.0,34.916000,32.0,27.010000,32.0
1084994,SS199706,5,11-MAY-1997 09:54:00,11-MAY-1997 10:43:00,0.0,990.0,4600.0,-14.69300,151.3260,-14.69300,...,-14.69300,151.3260,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,6.0,34.910000,32.0,26.983999,32.0
1084995,SS199706,5,11-MAY-1997 09:54:00,11-MAY-1997 10:43:00,0.0,990.0,4600.0,-14.69300,151.3260,-14.69300,...,-14.69300,151.3260,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,8.0,34.910000,32.0,26.985001,32.0
1084996,SS199706,5,11-MAY-1997 09:54:00,11-MAY-1997 10:43:00,0.0,990.0,4600.0,-14.69300,151.3260,-14.69300,...,-14.69300,151.3260,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,10.0,34.910000,32.0,26.989000,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085734,SS199706,5,11-MAY-1997 13:44:00,11-MAY-1997 14:20:00,0.0,504.0,4500.0,-14.61017,151.3685,-14.61017,...,-14.61017,151.3685,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,496.0,34.629002,32.0,8.964000,32.0
1085735,SS199706,5,11-MAY-1997 13:44:00,11-MAY-1997 14:20:00,0.0,504.0,4500.0,-14.61017,151.3685,-14.61017,...,-14.61017,151.3685,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,498.0,34.627998,32.0,8.925000,32.0
1085736,SS199706,5,11-MAY-1997 13:44:00,11-MAY-1997 14:20:00,0.0,504.0,4500.0,-14.61017,151.3685,-14.61017,...,-14.61017,151.3685,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,500.0,34.627998,32.0,8.909000,32.0
1085737,SS199706,5,11-MAY-1997 13:44:00,11-MAY-1997 14:20:00,0.0,504.0,4500.0,-14.61017,151.3685,-14.61017,...,-14.61017,151.3685,Tropical rock lobster assesment and modelling ...,4902.0,71d65ede-e21c-4527-acda-7d17093fbcd0,502.0,34.618000,32.0,8.853000,32.0


In [116]:
flag[np.logical_and(flag>= 0, flag<=63)] = 1
flag

masked_array(data=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                   1.0, 1.0, 1.0, 1.0,